In [1]:
import duckdb
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

import pandas as pd # Only used for date range and holiday generation


In [6]:
GOLD_VENTAS_WEEKLY_PATH = '../data/gold_ventas_semanales.parquet'
GOLD_FEATURE_SCALER_PATH = '../models/clustering/feature_scaler.joblib'
GOLD_FEATURES_PATH = '../models/clustering/gold_ventas_semanales_clustered.parquet'


In [2]:
def feature_engineering_clustering(table_path: str):
# --- 1. Preparar Datos de Entrada como PyArrow Table ---

    arrow_table = pq.read_table(table_path)


    print("--- Tabla Arrow de Entrada ---")
    print(arrow_table)
    print("\nEsquema:")
    print(arrow_table.schema)

    # --- 2. Conectar a DuckDB y Registrar la Tabla Arrow ---
    con = duckdb.connect(database=':memory:', read_only=False)

    # Registrar la tabla Arrow para que DuckDB pueda consultarla
    con.register('sales_data', arrow_table)
    print("--- Tabla Arrow Registrada en DuckDB ---")
    print(con.sql("SELECT * FROM sales_data LIMIT 5").fetch_arrow_table())
    # --- 3. Construir y Ejecutar la Consulta SQL para Extraer Features ---

    # Usaremos CTEs (Common Table Expressions) para organizar el cálculo, especialmente para ADI
    sql_query = """
    WITH WeeklyData AS (
        -- Convertir fecha a número de semana o algo secuencial y ordenable
        SELECT
            establecimiento,
            material,
            epoch(week) AS week_epoch, -- Usar epoch para cálculos de diferencia simples
            weekly_volume,
            has_promo,
            is_covid_period
        FROM sales_data
    ),
    SalesWeeks AS (
        -- Identificar las semanas con ventas para calcular ADI
        SELECT
            establecimiento,
            material,
            week_epoch
        FROM WeeklyData
        WHERE weekly_volume > 0
    ),
    LaggedSalesWeeks AS (
        -- Calcular la semana anterior con ventas para cada semana de venta
        SELECT
            establecimiento,
            material,
            week_epoch,
            LAG(week_epoch, 1) OVER (PARTITION BY establecimiento, material ORDER BY week_epoch) AS prev_sale_epoch
        FROM SalesWeeks
    ),
    Intervals AS (
        -- Calcular los intervalos en segundos entre ventas
        SELECT
            establecimiento,
            material,
            (week_epoch - prev_sale_epoch) AS interval_seconds
        FROM LaggedSalesWeeks
        WHERE prev_sale_epoch IS NOT NULL
    ),
    AdiAvg AS (
        -- Calcular el ADI promedio en segundos y convertir a semanas (aprox)
        SELECT
            establecimiento,
            material,
            AVG(interval_seconds) / (60*60*24*7) AS adi_weeks -- Segundos a semanas
        FROM Intervals
        GROUP BY establecimiento, material
    ),
    BaseFeatures AS (
        -- Calcular la mayoría de las features usando agregaciones estándar y condicionales
        SELECT
            establecimiento,
            material,
            -- Volumen / Magnitud
            SUM(weekly_volume) AS total_liters,
            AVG(weekly_volume) AS mean_liters,
            MEDIAN(weekly_volume) AS median_liters,
            MAX(weekly_volume) AS max_liters,
            STDDEV_SAMP(weekly_volume) AS std_liters, -- Desviación estándar muestral
            -- Intermitencia
            COUNT(*) AS num_weeks,
            COUNT(CASE WHEN weekly_volume > 0 THEN 1 ELSE NULL END) AS nonzero_weeks_count,
            AVG(CASE WHEN weekly_volume = 0 THEN 1.0 ELSE 0.0 END) AS zero_ratio,
            AVG(weekly_volume) FILTER (WHERE weekly_volume > 0) AS mean_nonzero_liters,
            MEDIAN(weekly_volume) FILTER (WHERE weekly_volume > 0) AS median_nonzero_liters,
            STDDEV_SAMP(weekly_volume) FILTER (WHERE weekly_volume > 0) AS std_nonzero_liters,
            -- Respuesta a Eventos
            AVG(weekly_volume) FILTER (WHERE has_promo = 1) AS mean_liters_promo,
            AVG(weekly_volume) FILTER (WHERE has_promo = 0) AS mean_liters_no_promo
            -- Agrega aquí cálculos para 'is_covid_period' si es necesario
        FROM WeeklyData
        GROUP BY establecimiento, material
    )
    -- Query Final: Unir features base con ADI y calcular CV2, Promo Lift
    SELECT
        bf.establecimiento,
        bf.material,
        bf.total_liters,
        bf.mean_liters,
        bf.median_liters,
        bf.max_liters,
        COALESCE(bf.std_liters, 0) as std_liters, -- Coalesce para stddev de grupos con 1 elemento
        bf.num_weeks,
        bf.nonzero_weeks_count,
        bf.zero_ratio,
        bf.mean_nonzero_liters,
        bf.median_nonzero_liters,
        COALESCE(bf.std_nonzero_liters, 0) as std_nonzero_liters,
        -- Calcular CV2 (manejar división por cero)
        CASE
            WHEN bf.mean_nonzero_liters IS NOT NULL AND bf.mean_nonzero_liters != 0
            THEN pow(COALESCE(bf.std_nonzero_liters, 0) / bf.mean_nonzero_liters, 2)
            ELSE NULL -- O 0 si prefieres
        END AS cv_squared,
        -- Unir ADI
        COALESCE(adi.adi_weeks, bf.num_weeks) AS adi, -- Si no hay ADI (pocas ventas), usar num_weeks como valor alto? O NULL?
        -- Calcular Promo Lift (manejar división por cero)
        CASE
            WHEN bf.mean_liters_no_promo IS NOT NULL AND bf.mean_liters_no_promo != 0
            THEN bf.mean_liters_promo / bf.mean_liters_no_promo
            ELSE NULL -- O 1 si no hay efecto medible o datos base
        END AS promo_lift
    FROM BaseFeatures bf
    LEFT JOIN AdiAvg adi
        ON bf.establecimiento = adi.establecimiento AND bf.material = adi.material
    ORDER BY bf.establecimiento, bf.material;
    """

    # Ejecutar la consulta y obtener el resultado como una Tabla Arrow
    features_arrow_table = con.execute(sql_query).arrow()

    # Cerrar conexión DuckDB
    con.close()

    print("\n--- Tabla Arrow con Features Agregadas (Antes de Escalar y Limpiar) ---")
    print(features_arrow_table)
    print("\nEsquema:")
    print(features_arrow_table.schema)

    features_arrow_table = fill_null_column(features_arrow_table, 'cv_squared', 0.0)
    features_arrow_table = fill_null_column(features_arrow_table, 'promo_lift', 1.0)
    features_arrow_table = fill_null_column(features_arrow_table, 'mean_nonzero_liters', 0.0)
    features_arrow_table = fill_null_column(features_arrow_table, 'median_nonzero_liters', 0.0)
    import pyarrow.compute as pc
    max_weeks = pc.max(features_arrow_table['num_weeks']).as_py()
    features_arrow_table = fill_null_column(features_arrow_table, 'adi', float(max_weeks))

    print("\n--- Tabla Arrow con Features Limpias ---")
    print(features_arrow_table.slice(0, 5)) # Mostrar primeras filas

    # --- 5. Escalar las Features usando NumPy y Scikit-learn ---

    # Nombres de las columnas de features a escalar
    feature_columns_for_clustering = [
        'total_liters', 'mean_liters', 'median_liters', 'max_liters', 'std_liters',
        'nonzero_weeks_count', 'zero_ratio', 'mean_nonzero_liters', 'median_nonzero_liters',
        'std_nonzero_liters', 'cv_squared', 'adi', 'promo_lift'
        # 'num_weeks' podría no ser necesaria si usas 'zero_ratio' o 'nonzero_weeks_count'
    ]
    # Asegurarse que todas las columnas seleccionadas existen
    feature_columns_for_clustering = [col for col in feature_columns_for_clustering if col in features_arrow_table.schema.names]


    # Extraer solo las columnas de features a un array NumPy
    feature_arrays = [features_arrow_table.column(col_name).to_numpy(zero_copy_only=False)
                    for col_name in feature_columns_for_clustering]
    features_numpy = np.stack(feature_arrays, axis=1)

    # Escalar
    scaler = StandardScaler()
    scaled_features_numpy = scaler.fit_transform(features_numpy)
    # Guardar el scaler
    import joblib
    import os

    scaler_path = GOLD_FEATURE_SCALER_PATH
    joblib.dump(scaler, scaler_path)
    print(f"Saved scaler to {scaler_path}")
    # --- 6. Crear la Tabla Arrow Final para Clustering ---

    # Crear nuevas columnas Arrow a partir de los arrays NumPy escalados
    scaled_arrow_cols = [pa.array(scaled_features_numpy[:, i]) for i in range(scaled_features_numpy.shape[1])]

    # Seleccionar las columnas identificadoras de la tabla original
    id_cols = [features_arrow_table.column('establecimiento'), features_arrow_table.column('material')]
    id_names = ['establecimiento', 'material']

    # Combinar identificadores y features escaladas en una nueva tabla
    final_arrow_table = pa.Table.from_arrays(
        id_cols + scaled_arrow_cols,
        names=id_names + feature_columns_for_clustering
    )

    print("\n--- Tabla Arrow Final Lista para Clustering (Features Escaladas) ---")
    print(final_arrow_table)
    print("\nEsquema Final:")
    print(final_arrow_table.schema)


    # Guardar como parquet file
    pq.write_table(final_arrow_table, GOLD_FEATURES_PATH)
    print(f"Saved features table to {GOLD_FEATURES_PATH}")

    # Para asegurarnos que se guarda correctamente, comprobamos si el archivo existe
    if os.path.exists(GOLD_FEATURES_PATH):
        print(f"Confirmed: Features file exists at {GOLD_FEATURES_PATH}")
    else:
        print(f"Warning: Features file was not created at {GOLD_FEATURES_PATH}")


    return features_arrow_table

    # --- 4. Limpieza de NULLs/NaNs en PyArrow (si es necesario) ---
    # DuckDB con COALESCE ya manejó algunos NULLs, pero podríamos tener otros
    # Por ejemplo, promo_lift puede ser NULL. Rellenamos con 1 (sin efecto)
    # cv_squared puede ser NULL. Rellenamos con 0 (sin varianza relativa)
    # mean/median_nonzero_liters pueden ser NULL si nunca hubo ventas. Rellenamos con 0.

def fill_null_column(table, col_name, fill_value):
    """Helper para rellenar NULLs en una columna de una tabla Arrow."""
    col_index = table.schema.get_field_index(col_name)
    if col_index == -1:
        return table # Columna no encontrada

    col = table.column(col_name)
    mask = pc.is_null(col)
    filled_col = pc.if_else(mask, pa.scalar(fill_value, type=col.type), col)
    return table.set_column(col_index, pa.field(col_name, filled_col.type), filled_col)




In [8]:
print("Starting feature engineering clustering...")
print(GOLD_VENTAS_WEEKLY_PATH)
processed_table = feature_engineering_clustering(GOLD_VENTAS_WEEKLY_PATH)

if processed_table:

    print("\n--- Processed Table Sample (First 10 rows) ---")
    # Convert to Pandas just for easy printing of the head
    print(processed_table.slice(0, 10).to_pandas())
    print("\n--- Processed Table Schema ---")
    print(processed_table.schema)
else:
    print("\nFeature engineering failed.")


Starting feature engineering clustering...
../data/gold_ventas_semanales.parquet
--- Tabla Arrow de Entrada ---
pyarrow.Table
establecimiento: string
material: string
week: date32[day]
has_promo: int32
weekly_volume: double
is_covid_period: int32
----
establecimiento: [["8100441311","8100441311","8100441311","8100441311","8100441311",...,"8100142319","8100142319","8100142319","8100142319","8100142319"],["8100142319","8100142319","8100142319","8100142319","8100142319",...,"8100093438","8100093438","8100093438","8100093438","8100093438"],...,["8100201874","8100201874","8100201874","8100201874","8100201874",...,"8100394850","8100394850","8100394850","8100394850","8100394850"],["8100394850","8100394850","8100394894","8100394894","8100394894",...,"8100023583","8100023583","8100023583","8100023583","8100023583"]]
material: [["ED13LT","ED13LT","ED13LT","ED13LT","ED13LT",...,"ED50","ED50","ED50","ED50","ED50"],["ED50","ED50","ED50","ED50","ED50",...,"ED50","ED50","ED50","ED50","ED50"],...,["VI

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [15]:
def calculate_silhouette_score(features_table):

    print("Esquema de la tabla cargada:")
    print(features_table.schema)

# --- 2. Preparar Datos para Scikit-learn ---

    # Nombres de las columnas de features (deben coincidir con las del archivo Parquet)
    # Asume que las primeras columnas son 'establecimiento', 'material' y el resto son features
    all_columns = features_table.schema.names
    identifier_columns = ['establecimiento', 'material'] # Ajusta si tienes otros IDs
    feature_columns = [col for col in all_columns if col not in identifier_columns]

    print(f"\nColumnas identificadoras: {identifier_columns}")
    print(f"Columnas de features para clustering: {feature_columns}")

    # Extraer solo las columnas de features a un array NumPy
    try:
        feature_arrays = [features_table.column(col_name).to_numpy(zero_copy_only=False)
                        for col_name in feature_columns]
        X_features = np.stack(feature_arrays, axis=1)
        print(f"Datos de features extraídos en array NumPy con forma: {X_features.shape}")

        # Verificar si hay NaNs o Infs residuales (importante para K-Means)
        if np.isnan(X_features).any() or np.isinf(X_features).any():
            print("\n¡Advertencia! Se encontraron valores NaN o Inf en las features."
                " K-Means fallará. Revisa el paso de limpieza/relleno de nulos.")
            # Opcional: intentar rellenar aquí, aunque es mejor hacerlo antes de guardar
            # X_features = np.nan_to_num(X_features, nan=0.0, posinf=0.0, neginf=0.0) # Ejemplo simple
            # print("Se intentó rellenar NaNs/Infs con 0.")
        else:
            print("No se encontraron NaNs/Infs en las features.")

    except Exception as e:
        print(f"Error al extraer features a NumPy: {e}")
        exit()


    # --- 3. Determinar el Número Óptimo de Clusters (k) ---

    # Rango de k a probar
    k_range = range(2, 11) # Probar de 2 a 10 clusters (ajusta según necesidad)
    inertia_values = []
    silhouette_scores = []

    print("\nCalculando métricas para determinar k óptimo...")
    for k in k_range:
        kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
        kmeans.fit(X_features)
        inertia_values.append(kmeans.inertia_) # Inertia: Suma de distancias al cuadrado al centroide (WCSS)
        # Silhouette score necesita al menos 2 clusters y puede ser lento en datos grandes
        if k >= 2:
            score = silhouette_score(X_features, kmeans.labels_)
            silhouette_scores.append(score)
            print(f"  k={k}, Inertia={kmeans.inertia_:.2f}, Silhouette Score={score:.4f}")
        else:
            silhouette_scores.append(np.nan) # No aplicable para k=1
            print(f"  k={k}, Inertia={kmeans.inertia_:.2f}")


    # --- Visualización para encontrar k ---

    plt.figure(figsize=(12, 5))

    # Gráfico del Codo (Elbow Method)
    plt.subplot(1, 2, 1)
    plt.plot(k_range, inertia_values, marker='o')
    plt.title('Método del Codo (Elbow Method)')
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Inertia (WCSS)')
    plt.xticks(list(k_range))
    plt.grid(True)

    # Gráfico del Coeficiente de Silueta
    plt.subplot(1, 2, 2)
    # Asegurarse de que k_range y silhouette_scores tengan la misma longitud si k_range empieza en 2
    valid_k_range_for_silhouette = [k for k in k_range if k >= 2]
    plt.plot(valid_k_range_for_silhouette, silhouette_scores, marker='o')
    plt.title('Coeficiente de Silueta')
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Silhouette Score Promedio')
    plt.xticks(valid_k_range_for_silhouette)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    print("\n--- Interpretación para elegir k ---")
    print(" - Método del Codo: Busca el 'codo' en la gráfica de Inertia, el punto donde la disminución se vuelve menos pronunciada.")
    print(" - Coeficiente de Silueta: Busca el valor de 'k' que maximiza el Silhouette Score (más cercano a 1 es mejor).")
    print("   Valores cercanos a 0 indican clusters solapados. Valores negativos indican que las muestras pueden estar en el cluster incorrecto.")
    print("Elige un valor de 'k' basado en estas gráficas y/o conocimiento del negocio.")



def train_clustering_model(features_table, k, transpose_view=False):
    all_columns = features_table.schema.names
    identifier_columns = ['establecimiento', 'material'] # Ajusta si tienes otros IDs
    feature_columns = [col for col in all_columns if col not in identifier_columns]

    feature_arrays = [features_table.column(col_name).to_numpy(zero_copy_only=False)
                      for col_name in feature_columns]
    X_features = np.stack(feature_arrays, axis=1)
    print(f"Datos de features extraídos en array NumPy con forma: {X_features.shape}")

    # Verificar si hay NaNs o Infs residuales (importante para K-Means)
    if np.isnan(X_features).any() or np.isinf(X_features).any():
        print("\n¡Advertencia! Se encontraron valores NaN o Inf en las features."
              " K-Means fallará. Revisa el paso de limpieza/relleno de nulos.")
        # Opcional: intentar rellenar aquí, aunque es mejor hacerlo antes de guardar
        # X_features = np.nan_to_num(X_features, nan=0.0, posinf=0.0, neginf=0.0) # Ejemplo simple
        # print("Se intentó rellenar NaNs/Infs con 0.")
    else:
        print("No se encontraron NaNs/Infs en las features.")



    # !!! CAMBIA ESTE VALOR BASADO EN EL ANÁLISIS ANTERIOR !!!
    print(f"\nEntrenando modelo K-Means final con k={k}...")

    kmeans_final = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
    kmeans_final.fit(X_features)

    # Obtener las etiquetas de cluster para cada punto de datos (cada serie)
    cluster_labels = kmeans_final.labels_

    print("Entrenamiento completado.")
    print(f"Número de puntos de datos por cluster: {np.bincount(cluster_labels)}")

    # --- 5. Añadir Etiquetas al Dataset Original y Guardar (Opcional) ---

    # Crear una columna Arrow con las etiquetas
    labels_column = pa.array(cluster_labels, type=pa.int32())

    # Añadir la columna de etiquetas a la tabla original (la que tiene los identificadores)
    # Nota: Esto asume que el orden de las filas en X_features coincide con features_table
    try:
        results_table = features_table.add_column(features_table.num_columns,
                                            pa.field('cluster_label', pa.int32()),
                                            labels_column)

        print("\n--- Tabla con Etiquetas de Cluster Añadidas (Primeras filas) ---")
        print(results_table.slice(0, 10))

        # Opcional: Guardar la tabla con etiquetas en un nuevo archivo Parquet
        # results_output_file = 'clustered_features.parquet'
        # pq.write_table(results_table, results_output_file)
        # print(f"\nTabla con etiquetas guardada en: {results_output_file}")

    except Exception as e:
        print(f"\nError al añadir/guardar etiquetas: {e}")
        # Si falla la adición directa, crear un mapeo y unirlo puede ser alternativa
        # Crear DataFrame de IDs y labels
        id_df = features_table.select(identifier_columns).to_pandas()
        id_df['cluster_label'] = cluster_labels
        print("\nMapeo de Identificadores a Cluster Labels (DataFrame Pandas):")
        print(id_df.head())
        # Aquí podrías unir este id_df con la tabla original si es necesario


    # --- 6. Analizar Centroides (Opcional pero útil) ---
    # Los centroides están en el espacio escalado. Para interpretarlos,
    # necesitarías el objeto 'scaler' del paso anterior para aplicar 'inverse_transform'.
    # Si no guardaste el scaler, solo puedes analizar los centroides escalados.
    print("\n--- Centroides de los Clusters (en espacio escalado) ---")
    centroids_scaled = kmeans_final.cluster_centers_
    for i, centroid in enumerate(centroids_scaled):
        print(f"\nCluster {i}:")
        for feature_name, value in zip(feature_columns, centroid):
            print(f"  {feature_name}: {value:.3f}")
    import pandas as pd
    # print as a table with feature_names as row names and values as column values
    print(pd.DataFrame(centroids_scaled))

    # Si tuvieras el scaler:
    try:
        from joblib import load
        import os
        scaler = load(os.path.join(config.DATA_DIR, "scaler", "features_scaler.joblib")) # Asume que guardaste el scaler
        centroids_original_scale = scaler.inverse_transform(centroids_scaled)
        print("\n--- Centroides de los Clusters (en escala original) ---")
        print(pd.DataFrame(centroids_original_scale))

        # Si se solicita la vista transpuesta
        if transpose_view:
            print("\n--- Generando tabla transpuesta de centroides ---")
            # Crear DataFrame transpuesto con nombres descriptivos de las features
            df_centroids = pd.DataFrame(
                centroids_original_scale.T,  # Transponemos la matriz
                index=feature_columns,  # Nombres de las features como índice (filas)
                columns=[f"Cluster {i}" for i in range(k)]  # Nombres de clusters como columnas
            )
            
            # Nombres más descriptivos para las features
            feature_descriptions = {
                'total_liters': 'Volumen Total (litros)',
                'mean_liters': 'Volumen Medio (litros/semana)',
                'median_liters': 'Volumen Mediana (litros/semana)',
                'max_liters': 'Volumen Máximo (litros/semana)',
                'std_liters': 'Desviación Estándar Volumen',
                'nonzero_weeks_count': 'Semanas con Venta',
                'zero_ratio': 'Ratio Semanas sin Venta',
                'mean_nonzero_liters': 'Volumen Medio en Semanas con Venta',
                'median_nonzero_liters': 'Mediana Volumen en Semanas con Venta',
                'std_nonzero_liters': 'Desv. Est. en Semanas con Venta',
                'cv_squared': 'Coeficiente Variación al Cuadrado',
                'adi': 'Intervalo Medio entre Demandas (ADI)',
                'promo_lift': 'Lift por Promoción'
            }
            
            # Reemplazar índices con descripciones más legibles
            df_centroids.index = [feature_descriptions.get(feat, feat) for feat in feature_columns]
            
            # Mostrar tabla transpuesta
            print("\n--- Tabla Transpuesta de Centroides (Escala Original) ---")
            pd.set_option('display.max_rows', None)  # Mostrar todas las filas
            pd.set_option('display.width', 120)  # Ancho suficiente para ver bien
            pd.set_option('display.precision', 2)  # Reducir decimales para legibilidad
            print(df_centroids)
            
            # Guardar como CSV para fácil acceso
            csv_path = os.path.join(config.DATA_DIR, "centroids_transposed.csv")
            df_centroids.to_csv(csv_path)
            print(f"\nTabla guardada en: {csv_path}")
            
            # Crear una visualización más atractiva con matplotlib
            plt.figure(figsize=(14, 8))
            
            # Crear tabla en matplotlib
            the_table = plt.table(
                cellText=df_centroids.round(2).values,
                rowLabels=df_centroids.index,
                colLabels=df_centroids.columns,
                loc='center',
                cellLoc='center'
            )
            
            # Ajustar tamaño y estilo
            the_table.auto_set_font_size(False)
            the_table.set_fontsize(9)
            the_table.scale(1.2, 1.5)
            
            # Eliminar ejes
            plt.axis('off')
            plt.title('Centroides de Clusters (Valores Transpuestos)', fontsize=16)
            
            # Guardar imagen
            plt_path = os.path.join(config.DATA_DIR, "centroids_transposed.png")
            plt.savefig(plt_path, bbox_inches='tight', dpi=150)
            print(f"Visualización guardada en: {plt_path}")
            plt.close()

    except FileNotFoundError:
        print("\nNo se encontró 'scaler.joblib'. No se pueden mostrar centroides en escala original.")


def transpose_cluster_centroids():
    """
    Genera una tabla transpuesta de los centroides de los clusters.
    Las filas serán los atributos/features y las columnas serán los clusters.
    """
    import pandas as pd
    from joblib import load
    import os
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    
    print("Generando tabla transpuesta de centroides...")
    
    # Cargar datos
    features_table = pq.read_table(config.GOLD_FEATURES_FULL_PATH)
    
    # Extraer nombres de features
    all_columns = features_table.schema.names
    identifier_columns = ['establecimiento', 'material']
    feature_columns = [col for col in all_columns if col not in identifier_columns]
    
    # Extraer datos para clustering
    feature_arrays = [features_table.column(col_name).to_numpy(zero_copy_only=False)
                      for col_name in feature_columns]
    X_features = np.stack(feature_arrays, axis=1)
    
    # Entrenar modelo KMeans con k=6 (o el número que se determinó como óptimo)
    k = 6
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
    kmeans.fit(X_features)
    centroids_scaled = kmeans.cluster_centers_
    
    # Cargar el scaler para convertir a escala original
    try:
        scaler = load(os.path.join(config.DATA_DIR, "scaler", "features_scaler.joblib"))
        centroids_original = scaler.inverse_transform(centroids_scaled)
        
        # Crear DataFrame transpuesto con nombres descriptivos de las features
        # Los nombres de las columnas serán "Cluster 0", "Cluster 1", etc.
        df_centroids = pd.DataFrame(
            centroids_original.T,  # Transponemos la matriz
            index=feature_columns,  # Nombres de las features como índice (filas)
            columns=[f"Cluster {i}" for i in range(k)]  # Nombres de clusters como columnas
        )
        
        # Nombres más descriptivos para las features
        feature_descriptions = {
            'total_liters': 'Volumen Total (litros)',
            'mean_liters': 'Volumen Medio (litros/semana)',
            'median_liters': 'Volumen Mediana (litros/semana)',
            'max_liters': 'Volumen Máximo (litros/semana)',
            'std_liters': 'Desviación Estándar Volumen',
            'nonzero_weeks_count': 'Semanas con Venta',
            'zero_ratio': 'Ratio Semanas sin Venta',
            'mean_nonzero_liters': 'Volumen Medio en Semanas con Venta',
            'median_nonzero_liters': 'Mediana Volumen en Semanas con Venta',
            'std_nonzero_liters': 'Desv. Est. en Semanas con Venta',
            'cv_squared': 'Coeficiente Variación al Cuadrado',
            'adi': 'Intervalo Medio entre Demandas (ADI)',
            'promo_lift': 'Lift por Promoción'
        }
        
        # Reemplazar índices con descripciones más legibles
        df_centroids.index = [feature_descriptions.get(feat, feat) for feat in feature_columns]
        
        # Mostrar tabla transpuesta
        print("\n--- Tabla Transpuesta de Centroides (Escala Original) ---")
        pd.set_option('display.max_rows', None)  # Mostrar todas las filas
        pd.set_option('display.width', 120)  # Ancho suficiente para ver bien
        pd.set_option('display.precision', 2)  # Reducir decimales para legibilidad
        print(df_centroids)
        
        # Guardar como CSV para fácil acceso
        csv_path = os.path.join(config.DATA_DIR, "centroids_transposed.csv")
        df_centroids.to_csv(csv_path)
        print(f"\nTabla guardada en: {csv_path}")
        
        # Crear una visualización más atractiva con matplotlib
        plt.figure(figsize=(14, 8))
        
        # Crear tabla en matplotlib
        the_table = plt.table(
            cellText=df_centroids.round(2).values,
            rowLabels=df_centroids.index,
            colLabels=df_centroids.columns,
            loc='center',
            cellLoc='center'
        )
        
        # Ajustar tamaño y estilo
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(9)
        the_table.scale(1.2, 1.5)
        
        # Eliminar ejes
        plt.axis('off')
        plt.title('Centroides de Clusters (Valores Transpuestos)', fontsize=16)
        
        # Guardar imagen
        plt_path = os.path.join(config.DATA_DIR, "centroids_transposed.png")
        plt.savefig(plt_path, bbox_inches='tight', dpi=150)
        print(f"Visualización guardada en: {plt_path}")
        plt.close()
        
    except FileNotFoundError:
        print("\nNo se encontró el archivo scaler. No se puede generar la tabla transpuesta.")
    except Exception as e:
        print(f"\nError al generar tabla transpuesta: {e}")

# Función para asignar clusters a los datos de entrenamiento
def assign_clusters_to_training_data(k: int):
    """
    Función principal que asigna los clusters a los datos de entrenamiento.
    """
    print("Asignando clusters a los datos de entrenamiento...")
    
    # 1. Cargar datos de entrenamiento
    training_data = pq.read_table(config.GOLD_WEEKLY_FULL_PATH)
    print(f"Datos de entrenamiento cargados desde {config.GOLD_WEEKLY_FULL_PATH}")
    print(f"Número de registros: {training_data.num_rows}")
    
    # 2. Cargar datos de features (que contienen los identificadores y las features para clustering)
    features_table = pq.read_table(config.GOLD_FEATURES_FULL_PATH)
    print(f"Datos de features cargados desde {config.GOLD_FEATURES_FULL_PATH}")
    print(f"Número de registros: {features_table.num_rows}")
    
    # 3. Extraer identificadores y features para clustering
    all_columns = features_table.schema.names
    identifier_columns = ['establecimiento', 'material']
    feature_columns = [col for col in all_columns if col not in identifier_columns]
    
    # 4. Extraer datos para clustering
    feature_arrays = [features_table.column(col_name).to_numpy(zero_copy_only=False)
                    for col_name in feature_columns]
    X_features = np.stack(feature_arrays, axis=1)
    
    # 5. Ejecutar clustering (mismo modelo, mismos parámetros que en 04_clustering_model.py)
    
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto', random_state=42)
    kmeans.fit(X_features)
    
    # 6. Obtener etiquetas
    cluster_labels = kmeans.labels_
    
    # 7. Crear un DataFrame con identificadores y etiquetas de cluster
    id_df = features_table.select(identifier_columns).to_pandas()
    id_df['cluster_label'] = cluster_labels
    
    # 8. Convertir los datos de entrenamiento a DataFrame para facilitar el merge
    training_df = training_data.to_pandas()
    
    # 9. Combinar los datos de entrenamiento con las etiquetas de cluster
    # Usamos merge en lugar de join para mantener todos los registros
    merged_df = pd.merge(
        training_df, 
        id_df, 
        on=['establecimiento', 'material'],
        how='left'
    )
    
    # 10. Verificar que todos los registros han sido asignados a un cluster
    null_clusters = merged_df['cluster_label'].isna().sum()
    if null_clusters > 0:
        print(f"ADVERTENCIA: {null_clusters} registros no tienen asignado un cluster.")
        # Rellenar valores nulos con un valor que indique "no clasificado" (-1)
        merged_df['cluster_label'] = merged_df['cluster_label'].fillna(-1).astype(int)
    
    # 11. Convertir de nuevo a tabla Arrow
    result_table = pa.Table.from_pandas(merged_df)
    
    # 12. Guardar el resultado
    output_path = os.path.join(config.DATA_DIR, "gold_ventas_semanales_training_clustered.parquet")
    pq.write_table(result_table, output_path)
    print(f"Datos con clusters guardados en: {output_path}")
    
    # 13. Mostrar distribución de clusters
    cluster_counts = merged_df['cluster_label'].value_counts().sort_index()
    print("\nDistribución de registros por cluster:")
    for cluster, count in cluster_counts.items():
        print(f"Cluster {cluster}: {count} registros ({count/len(merged_df)*100:.2f}%)")
    
    return output_path




In [12]:
print("\nCargando datos de features para análisis de k...")
try:
    features_path = GOLD_FEATURES_PATH
    features_table_for_k = pq.read_table(GOLD_FEATURES_PATH)
    print(features_table_for_k.slice(0,5))
    print(f"Datos cargados desde {features_path}")
    # imprimir materiales unicos

    # Llamar a la función que calcula y muestra las gráficas
    calculate_silhouette_score(features_table_for_k)

except FileNotFoundError:
    print(f"Error: No se encontró el archivo de features en {features_path}")
    print("No se puede proceder con el análisis de k.")
except Exception as e:
    print(f"Error durante el análisis de k: {e}")

# --- FIN DEL SCRIPT (en esta modificación) ---
# El código se detendrá aquí después de mostrar las gráficas.
# Para ejecutar los siguientes pasos (entrenamiento final, asignación),
# necesitarías:
# 1. Analizar las gráficas mostradas y ELEGIR un valor para 'k'.
# 2. Modificar este bloque __main__ o crear otro script para llamar a:
#    - train_clustering_model(features_table, k_elegido, transpose_view=True)
#    - assign_clusters_to_training_data(k_elegido)
#    (Asegúrate de cargar 'features_table' de nuevo si es necesario para esas funciones)

print("\n--- SCRIPT FINALIZADO ---")
print("Revisa las gráficas mostradas (Codo y Silueta) para elegir el valor óptimo de 'k'.")
print("Luego, modifica el script o ejecuta las funciones correspondientes")
print("con el 'k' elegido para entrenar el modelo final y/o asignar clusters.")




Cargando datos de features para análisis de k...
pyarrow.Table
establecimiento: string
material: string
total_liters: double
mean_liters: double
median_liters: double
max_liters: double
std_liters: double
nonzero_weeks_count: double
zero_ratio: double
mean_nonzero_liters: double
median_nonzero_liters: double
std_nonzero_liters: double
cv_squared: double
adi: double
promo_lift: double
----
establecimiento: [["8100000005","8100000005","8100000005","8100000006","8100000006"]]
material: [["DL13","ED13","FDT13","ED13","FD13"]]
total_liters: [[-0.2715087116995092,-0.2603856571892176,-0.2614979626402467,-0.07685525776940644,-0.249262602678926]]
mean_liters: [[-0.25617493255741863,-0.26653647342670717,-0.22337633839742496,0.18862518125395455,-0.16421625734248307]]
median_liters: [[-0.27636592137944144,-0.27636592137944144,-0.27636592137944144,0.5712362332950929,-0.27636592137944144]]
max_liters: [[-0.18195482062772375,-0.13297433305246037,-0.18195482062772375,-0.05134018709368802,-0.165627991

In [17]:
k = 6
features_table = pq.read_table(GOLD_FEATURES_PATH)
# Puedes descomentar una de estas funciones según lo que quieras hacer
# Opción 1: Ejecutar clustering y generar vista transpuesta
train_clustering_model(features_table, k, transpose_view=True)

# Opción 2: Asignar clusters a los datos de entrenamiento
assign_clusters_to_training_data(k)


Datos de features extraídos en array NumPy con forma: (306404, 13)
No se encontraron NaNs/Infs en las features.

Entrenando modelo K-Means final con k=6...
Entrenamiento completado.
Número de puntos de datos por cluster: [217053  13918   4520      4     47  70862]

--- Tabla con Etiquetas de Cluster Añadidas (Primeras filas) ---
pyarrow.Table
establecimiento: string
material: string
total_liters: double
mean_liters: double
median_liters: double
max_liters: double
std_liters: double
nonzero_weeks_count: double
zero_ratio: double
mean_nonzero_liters: double
median_nonzero_liters: double
std_nonzero_liters: double
cv_squared: double
adi: double
promo_lift: double
cluster_label: int32
----
establecimiento: [["8100000005","8100000005","8100000005","8100000006","8100000006","8100000006","8100000006","8100000006","8100000009","8100000009"]]
material: [["DL13","ED13","FDT13","ED13","FD13","FDL13","FDT13","VI13","DL13","ED13"]]
total_liters: [[-0.2715087116995092,-0.2603856571892176,-0.26149796

NameError: name 'config' is not defined